In [1]:
import os
import tensorflow as tf

home = os.environ['HOME']
modeldir = os.path.join(home, 'YogaModel/src')

modelfile = os.path.join(modeldir, 'YogaModelTransferLearningFineTune.h5')
load_model = tf.keras.models.load_model(modelfile)
load_model.summary()

2021-11-18 21:52:27.446959: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-18 21:52:27.447850: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0     

In [2]:
home = os.environ['HOME']
saveddir = os.path.join(home, 'YogaModel/SavedModel/YogaModel/1')
tf.saved_model.save(load_model, saveddir)

2021-11-18 22:00:25.476240: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/jupyter/YogaModel/SavedModel/YogaModel/1/assets


In [3]:
converter = tf.lite.TFLiteConverter.from_saved_model(saveddir)
tflite_model = converter.convert()

2021-11-18 22:02:29.255836: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-11-18 22:02:29.255893: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2021-11-18 22:02:29.255904: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored change_concat_input_ranges.
2021-11-18 22:02:29.258211: I tensorflow/cc/saved_model/reader.cc:38] Reading SavedModel from: /home/jupyter/YogaModel/SavedModel/YogaModel/1
2021-11-18 22:02:29.309223: I tensorflow/cc/saved_model/reader.cc:90] Reading meta graph with tags { serve }
2021-11-18 22:02:29.309276: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/jupyter/YogaModel/SavedModel/YogaModel/1
2021-11-18 22:02:29.309455: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for be

In [4]:
import pathlib
tflite_model_file = pathlib.Path('YogaModel.tflite')
tflite_model_file.write_bytes(tflite_model)

9415460

In [5]:
import pathlib

home = os.environ['HOME']
tfliteDir = os.path.join(home, 'YogaModel/TFLiteModel')

tfLiteFile = os.path.join(tfliteDir, 'YogaModel.tflite')



In [6]:
interpreter = tf.lite.Interpreter(model_path=tfLiteFile)
interpreter.allocate_tensors()

In [7]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()



[{'name': 'serving_default_input_2:0', 'index': 0, 'shape': array([  1, 160, 160,   3], dtype=int32), 'shape_signature': array([ -1, 160, 160,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 180, 'shape': array([  1, 107], dtype=int32), 'shape_signature': array([ -1, 107], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [8]:
print(input_details)

[{'name': 'serving_default_input_2:0', 'index': 0, 'shape': array([  1, 160, 160,   3], dtype=int32), 'shape_signature': array([ -1, 160, 160,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [9]:
print(output_details)

[{'name': 'StatefulPartitionedCall:0', 'index': 180, 'shape': array([  1, 107], dtype=int32), 'shape_signature': array([ -1, 107], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [10]:
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]



In [18]:
PATH = '/home/jupyter/YogaModel/dataset/'
train_dir = os.path.join(PATH, 'training')
test_dir = os.path.join(PATH, 'testing')
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

Found 4752 files belonging to 107 classes.
Found 1241 files belonging to 107 classes.


In [29]:

image_batch, label_batch = validation_dataset.as_numpy_iterator().next()


2021-11-18 22:47:28.569141: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


In [30]:
predictions = []
test_labels, test_imgs = [], []

In [31]:
for i in range(9):
    img = image_batch[i]
    label = label_batch[i]
    
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    interpreter.set_tensor(input_index, img_array)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))
    test_labels.append(label)
    test_imgs.append(img)

In [32]:
import numpy as np
for i in range(9):
    prediction = predictions[i]
    score = tf.nn.softmax(prediction[0])
    label = test_labels[i]
    print ("Actual Label: "+ str(label))
    predicted_label = np.argmax(score)
    print ("predicted_label : "+ str(predicted_label))

Actual Label: 47
predicted_label : 52
Actual Label: 66
predicted_label : 66
Actual Label: 8
predicted_label : 20
Actual Label: 96
predicted_label : 96
Actual Label: 104
predicted_label : 101
Actual Label: 101
predicted_label : 101
Actual Label: 69
predicted_label : 87
Actual Label: 27
predicted_label : 105
Actual Label: 78
predicted_label : 78
